In [1]:
from dataloader.BSD500 import BSD500
import torch
from R_network_relax import RNet
import json
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

def compute_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = compare_psnr(Iclean[0,:,:,:], Img[0,:,:,:], data_range=data_range)
    return PSNR
    

In [3]:
test_dataset = BSD500('data/test.h5')

In [4]:
sigma = 25  


val_data = pd.read_csv(f"{'denoise_val_results'}/validation_scores_{'sigma_25_relaxed_valid'}.csv").reset_index(drop=True)
p1 = val_data.loc[val_data["psnr"].idxmax()]["p1"]


exp = "/home/pourya/mm_final/exps/sigma25/64_2_7_1e-3_relaxed"
path_ckp = exp + "/checkpoints/checkpoint_best_epoch.pth"
path_config = exp + "/config.json"
device = 'cuda:0'
config = json.load(open(path_config))
ckp = torch.load(path_ckp, map_location={'cuda:0':device,'cuda:1':device,'cuda:2':device,'cuda:3':device})

model = RNet(config['model_params'])
model.to(device)
model.load_state_dict(ckp['state_dict'])
model.eval()


model.eigenimage = model.eigenimage.to(device)
lip, model.eigenimage = model.W1.cal_lip(model.eigenimage, 1000)
model.alpha = 1 / lip
model.lmbda = torch.nn.Parameter(torch.ones(1, 1).to(device) * p1)


model.W1.W1.weight.data = model.zeromean(model.W1.W1.weight)
for i, _ in enumerate(model.W1.W1s): 
    model.W1.W1s[i].weight.data = model.zeromean(model.W1.W1s[i].weight)

In [5]:
def safi_denoise(y):
    s = 1e-3
    n = 5
    e = 1e-5
    q = (e/s) ** (1/n)
    tols = [s * q**(i) for i in range(n+1)]
    n_out = 10
    for _ in range(n_out-(n+1)):
        tols.append(e)
    with torch.no_grad(): 
        n_in = 100  
        c_k = model.prox_denoise_no_mask(y, y, n_in, tols[0], check_tol=True)
        for it in range(n_out - 1):   
            model.cal_mask(c_k)
            c_k_new = model.prox_denoise_with_mask(y, c_k, n_in, tols[it+1], check_tol=True)
            rel_err_out = (torch.norm(c_k - c_k_new) / torch.norm(c_k)).item() 
            sc = "{:0.1e}".format(rel_err_out)
            c_k = c_k_new
    
    return c_k

In [6]:
i = 0
psnrs = list()

with torch.no_grad():
    for img in test_dataset:
        i = i + 1
        if True:
            gt = img.to(device)[None, :, :, :]
            noisy_image = (img.to(device) + (25/255.0)*torch.randn(img.shape, device=device))[None, :, :, :]
            denoised =safi_denoise(noisy_image)
            psnr = np.round(compute_PSNR(denoised, gt, 1), 2)
            psnrs.append(psnr)
            print(i, psnr)

1 29.93
2 28.56
3 30.55
4 29.79
5 30.02
6 26.5
7 28.93
8 28.64
9 29.82
10 28.9
11 28.25
12 26.16
13 29.47
14 26.56
15 28.16
16 30.79
17 28.32
18 29.32
19 31.45
20 29.56
21 33.08
22 27.44
23 26.67
24 24.37
25 27.89
26 28.81
27 27.87
28 32.87
29 26.06
30 30.34
31 27.86
32 32.02
33 30.01
34 23.96
35 28.57
36 28.26
37 29.03
38 27.49
39 33.58
40 26.3
41 30.81
42 32.77
43 30.96
44 30.25
45 27.05
46 29.68
47 28.86
48 26.85
49 25.84
50 38.39
51 27.44
52 33.6
53 33.63
54 30.11
55 29.01
56 27.19
57 26.23
58 27.5
59 24.48
60 26.57
61 28.51
62 28.05
63 30.99
64 24.69
65 28.85
66 32.92
67 32.88
68 29.44


In [11]:
print(np.round(np.mean(np.array(psnrs)), 2), np.round(np.std(np.array(psnrs)), 2))

29.05 2.57
